# Daily Performance
Identify whether it is volatility or price movement that needs improvement

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error

In [36]:
file = "dailyPerf_2022-04-02_2022-04-05.csv"

In [37]:
df = pd.read_csv(file)
df.head()

,Date,Symbol,Exchange,garch,svr,mlp,LSTM,prev_Close,prediction,volatility,ActualDate,ActualClose,ActualPercent,ActualTrend
0,2022-04-04,AAL,NASDAQ,9.28532,2.47927,2.28121,16.6536,18.23,-1,4.68193,2022-04-05,17.84,2.139331,-1
1,2022-04-04,AAPL,NASDAQ,2.79687,1.75485,1.60878,159.0210,178.44,-1,2.05350,2022-04-05,175.06,1.894194,0
2,2022-04-04,AMAT,NASDAQ,6.81057,3.11612,3.11230,131.5450,129.44,1,4.34633,2022-04-05,121.71,5.971879,-1
3,2022-04-04,AMC,NYSE,101.91800,5.43978,5.71697,16.0923,23.31,-1,37.69170,2022-04-05,21.21,9.009009,-1
4,2022-04-04,AMD,NASDAQ,16.30120,4.39617,4.43730,110.4540,110.53,-1,8.37822,2022-04-05,106.82,3.356555,-1


In [93]:
df['new_garch'] = df.garch ** 0.5

In [38]:
df['LSTM_vol'] = (df.LSTM - df.prev_Close) / df.prev_Close * 100

## Which Voltatily metrics is closest

In [94]:
mean_squared_error(df.garch, df.ActualPercent)

190.61142176108277

In [89]:
mean_squared_error(df.garch ** 0.5, df.ActualPercent)

1.4480299679926973

In [40]:
mean_squared_error(df.svr, df.ActualPercent)

3.2556234171227607

In [41]:
mean_squared_error(df.mlp, df.ActualPercent)

2.8191002436477577

In [42]:
mean_squared_error(df.LSTM_vol, df.ActualPercent)

142.02571278119512

## New Aggregation

In [96]:
df["new_vol"] = df[['new_garch', 'svr', 'mlp']].mean(axis=1) # ML

In [97]:
mean_squared_error(df.new_vol, df.ActualPercent)

2.0945577462971112

In [98]:
def get_price_movement(change):
    if change > 0:
        return 1
    elif change < 0:
        return -1
    else:
        return 0

def get_above_threshold(volatility, threshold):
    if volatility > threshold:
        return True
    else:
        return False

def get_prediction(df):
    if df["new_price_movement"] == 1 and df["new_above_threshold"]:
        return 1 
    elif df["new_price_movement"] == -1 and df["new_above_threshold"]:
        return -1 
    else:
        return 0

In [101]:
df["new_price_movement"] = (df.LSTM - df.prev_Close).apply(get_price_movement)
threshold = 2
df["new_above_threshold"] = df.new_vol.apply(lambda x: get_above_threshold(x, threshold))
df["new_prediction"] = df.apply(get_prediction, axis=1)

In [102]:
df.head()

,Date,Symbol,Exchange,garch,svr,mlp,LSTM,prev_Close,prediction,volatility,...,ActualPercent,ActualTrend,LSTM_vol,new_vol,new_price_movement,new_above_threshold,new_prediction,actual_price_movement,actual_above_threshold,new_garch
0,2022-04-04,AAL,NASDAQ,9.28532,2.47927,2.28121,16.6536,18.23,-1,4.68193,...,2.139331,-1,-8.647285,2.602554,-1,True,-1,-1,True,3.047182
1,2022-04-04,AAPL,NASDAQ,2.79687,1.75485,1.60878,159.0210,178.44,-1,2.05350,...,1.894194,0,-10.882650,1.678672,-1,False,0,-1,False,1.672385
2,2022-04-04,AMAT,NASDAQ,6.81057,3.11612,3.11230,131.5450,129.44,1,4.34633,...,5.971879,-1,1.626236,2.946042,1,True,1,-1,True,2.609707
3,2022-04-04,AMC,NYSE,101.91800,5.43978,5.71697,16.0923,23.31,-1,37.69170,...,9.009009,-1,-30.963964,7.084065,-1,True,-1,-1,True,10.095445
4,2022-04-04,AMD,NASDAQ,16.30120,4.39617,4.43730,110.4540,110.53,-1,8.37822,...,3.356555,-1,-0.068760,4.290315,-1,True,-1,-1,True,4.037474


## Price Movement

In [103]:
mean_squared_error(df.LSTM, df.ActualClose)

559.2308316781093

In [104]:
df["actual_price_movement"] = (df.ActualClose - df.prev_Close).apply(get_price_movement)

In [105]:
accuracy_score(df.actual_price_movement, df.new_price_movement, normalize=True)

0.6794871794871795

In [106]:
f1_score(y_true=df.actual_price_movement, y_pred=df.new_price_movement, average='macro')

0.4705403203909856

In [107]:
f1_score(y_true=df.actual_price_movement, y_pred=df.new_price_movement, average='micro')

0.6794871794871795

In [108]:
f1_score(y_true=df.actual_price_movement, y_pred=df.new_price_movement, average='weighted')

0.7008082875582196

## Above Threshold

In [109]:
df["actual_above_threshold"] = df.ActualPercent.apply(lambda x: get_above_threshold(x, threshold))

In [110]:
accuracy_score(df.actual_above_threshold, df.new_above_threshold, normalize=True)

0.782051282051282

In [111]:
f1_score(y_true=df.actual_above_threshold, y_pred=df.new_above_threshold, average='macro')

0.7817283950617284

In [112]:
f1_score(y_true=df.actual_above_threshold, y_pred=df.new_above_threshold, average='micro')

0.782051282051282

In [113]:
f1_score(y_true=df.actual_above_threshold, y_pred=df.new_above_threshold, average='weighted')

0.7815131370686925

## Performance Metrics

In [114]:
accuracy_score(df.ActualTrend, df.new_prediction, normalize=True)

0.6794871794871795

In [115]:
f1_score(y_true=df.ActualTrend, y_pred=df.new_prediction, average='macro')

0.4754956977179199

In [116]:
f1_score(y_true=df.ActualTrend, y_pred=df.new_prediction, average='micro')

0.6794871794871795

In [117]:
f1_score(y_true=df.ActualTrend, y_pred=df.new_prediction, average='weighted')

0.694955250510806